<a href="https://colab.research.google.com/github/Hyun-Jun-Lee/Review_Sentiment_Analysis/blob/main/Movie_Review_Seniment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [ ]:
!pip install konlpy

In [1]:
import pandas as pd
import pymongo
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split


# Read Data

In [2]:
df = pd.read_csv('/content/drive/MyDrive/crawling_review.csv')
stopwords = pd.read_csv('/content/drive/MyDrive/stopword.csv')
stopwords = list(stopwords['stopdword'].values)

In [5]:
df.sample(10)

,Unnamed: 0,star,comment,label
322,338,9,앤드류 가필드 연기가 정말 좋다 조나단 라슨에 대해 하나도 모르고 봤는데 재밌었다 ...,1
1477,1593,10,음스러운 장면도 있었지만 재밌게 잘 봤다,1
871,943,10,이 영화 제 인생 최고의 영화입니다,1
8548,9051,6,이것도 미화 아닌가,0
376,395,10,대박 꿀잼이에요 저희 학교 단체로 봤는데 애들 다 만족했어요 ㅠㅠㅠ 다들보새요,1
8924,9435,4,지루하고 단조로우면서 재미없네요 마무리마저 미국식 신파라니,0
1666,1795,10,명작인데 평점점 뭐냐 ㅋ,1
3143,3378,7,재밌게 봤어요 편도 보고 싶어지네요,1
32,37,10,다양한 로맨스를 무겁지 않고 유쾌하게 그린 영화에요 역할마다 특색있고 재미있습니다,1
547,581,8,마지막 억지 신파만 아니었으면 점 주고 싶은 영화 차라리 주요 캐릭터들 살려서 속편...,1


In [3]:
stopwords[:10]

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라']

In [4]:
# NA값 제거
df = df.dropna(axis=0)
df.isna().sum()

Unnamed: 0    0
star          0
comment       0
label         0
dtype: int64

In [5]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [6]:
df = df.sample(frac=1).reset_index()
df.tail()

,index,star,comment,label
9457,7675,2,경제학도로서 보다가 포기했다,0
9458,5573,10,내가 처음으로 어머니랑 극장가서 본 영화 이 때 기억이 너무 강렬해서 지금 살인데도...,1
9459,4479,10,올해 나온 한국 영화중 가장 군계일학 스토리가 미쳤다,1
9460,7400,4,진짜 김정호의 스토리로 만들었다면 명작이었을 텐데 왜곡과 과장이 아주 심해서 몰입감...,0
9461,7096,5,곤지암이랑 느낌이 비슷한데 소리 엄청 질러댐,0


In [7]:
df.drop('index', axis=1, inplace=True)

In [8]:
df

,star,comment,label
0,10,유오성이 연기의 신인거 다시한번 인정스토리 몰입감 장난아님 남편 손 꼭잡고 숨도 못...,1
1,6,노인 극혐이다 징짜 아오,0
2,8,재밌네요 새로운 재미를 느꼈어요,1
3,10,기대안하고 봤는데 나오신 연기자분들 다들 연기 잘하시고요 액션이 박진감이 넘쳐서 시...,1
4,10,하 윤계상 사랑해여 존잼액션 좋아하시는 분들 꼭 보세요,1
...,...,...,...
9457,2,경제학도로서 보다가 포기했다,0
9458,10,내가 처음으로 어머니랑 극장가서 본 영화 이 때 기억이 너무 강렬해서 지금 살인데도...,1
9459,10,올해 나온 한국 영화중 가장 군계일학 스토리가 미쳤다,1
9460,4,진짜 김정호의 스토리로 만들었다면 명작이었을 텐데 왜곡과 과장이 아주 심해서 몰입감...,0


In [9]:
# train_test_split
target= 'label'
features = df.drop(columns= target).columns

train, test = train_test_split(df, test_size=0.2, random_state=2021, stratify=df[target])

In [10]:
print(len(train), len(test))

7569 1893


# Preprocessing

## 토큰화 및 stopword 제거

In [ ]:
!pip install jpype1==0.7.0

In [11]:
# 한국 형태소 분석기
okt = Okt()

In [12]:
X_train = []
for sentence in tqdm(train['comment']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)

100%|██████████| 7569/7569 [00:32<00:00, 231.94it/s]


In [13]:
X_test = []
for sentence in tqdm(test['comment']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

100%|██████████| 1893/1893 [00:05<00:00, 332.58it/s]


## 정수 인코딩

In [14]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [21]:
# 높은 정수가 부여된 단어가 등장 빈도수가 낮음을 의미
print(tokenizer.word_index)

{'하다': 1, '보다': 2, '영화': 3, '도': 4, '좋다': 5, '는': 6, '은': 7, '너무': 8, '이다': 9, '없다': 10, '연기': 11, '재밌다': 12, '배우': 13, '적': 14, '되다': 15, '한': 16, '스토리': 17, '만': 18, '다': 19, '않다': 20, '진짜': 21, '아니다': 22, '생각': 23, '나오다': 24, '자다': 25, '인': 26, '게': 27, '수': 28, '안': 29, '더': 30, '정말': 31, '연출': 32, '만들다': 33, '사람': 34, '감동': 35, '보고': 36, '가다': 37, '하고': 38, '볼': 39, '못': 40, '님': 41, '고': 42, '감독': 43, '그냥': 44, '최고': 45, '요': 46, '싶다': 47, '들다': 48, '모르다': 49, '액션': 50, '재미있다': 51, '아쉽다': 52, '느낌': 53, '내용': 54, '거': 55, '잘': 56, '말': 57, '내': 58, '이렇다': 59, '평점': 60, '점': 61, '마지막': 62, '오다': 63, '작품': 64, '많다': 65, '그렇다': 66, '중': 67, '넘다': 68, '재미': 69, '다시': 70, '지루하다': 71, '이야기': 72, '서': 73, '장면': 74, 'ㅠㅠ': 75, '정도': 76, '매력': 77, '많이': 78, '없이': 79, '몰입': 80, '지': 81, '사랑': 82, '성': 83, '주인공': 84, '영상': 85, '상미': 86, 'ㅋㅋ': 87, '꼭': 88, '처음': 89, '기대': 90, '아깝다': 91, '알다': 92, '뭐': 93, '역시': 94, '끝': 95, '듯': 96, '두': 97, '보기': 98, '나다': 99, '편': 100, '주다': 101, '분': 102, '

In [22]:
threshold = 2
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 10619
등장 빈도가 1번 이하인 희귀 단어의 수: 5106
단어 집합에서 희귀 단어의 비율: 48.08362369337979
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 4.6789091708819
